In [1]:
# Pega um RN já treinada e classifica a imagem 

In [1]:
import os
os.chdir('/mnt/mint_files/TCC/main_code/')

In [2]:
import numpy as np
from tensorflow import keras
from glob import glob
import rasterio as rio
from pyrsgis import raster
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from PIL import Image as im

2023-05-22 02:04:59.844102: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-22 02:04:59.844120: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
# Lê as imagens do teste

recreioFile = 'images/lagoa_freitas/2015/lagoa_freitas_v1_B[1-6].TIF'
#recreioFile = 'images/lagoa_freitas/land_5/2011/lagoa_freitas_v1_B[1-6].tif' 
recreio_bands = glob(recreioFile)
recreio_bands.sort()

#recreioFile = [
#    'images/barra/2015/barra_B4.TIF',
#    'images/barra/2015/barra_B3.TIF',
#    'images/barra/2015/barra_B2.TIF',
#]
#recreio_bands = recreioFile

l = []
for i in recreio_bands:
    with rio.open(i, 'r') as f:
        l.append(f.read(1))

# Data
arr_st = np.stack(l)

txt = 'Imagem shape D(Bands: {0}, Height: {1}, Width: {2})'.format(arr_st.shape[0], arr_st.shape[1], arr_st.shape[2])
print(txt)

Imagem shape D(Bands: 6, Height: 360, Width: 500)


In [4]:
# reshape as bandas da imagem de teste
x = np.moveaxis(arr_st, 0, -1)
x.reshape(-1, arr_st.shape[0]).shape, arr_st.shape[1] * arr_st.shape[2]
X_data = x.reshape(-1, arr_st.shape[0])
scaler = StandardScaler().fit(X_data)
X_scaled = scaler.transform(X_data)

X_scaled


array([[ 0.87993796,  0.93574309,  0.92082308,  1.00759047,  0.33292613,
         0.93900401],
       [ 1.50115632,  1.6334487 ,  1.54248644,  1.61688701,  0.41911844,
         1.31424864],
       [ 1.29408354,  1.38904693,  1.3783509 ,  1.48586414,  0.48862836,
         1.50906476],
       ...,
       [ 1.64644126,  1.06408816,  0.71257612,  0.38543279, -0.69999131,
        -0.60824821],
       [ 1.68317999,  1.16649113,  0.77002356,  0.45777671, -0.67413362,
        -0.56080795],
       [ 1.86854353,  1.28254784,  0.80387651,  0.4497385 , -0.66384615,
        -0.50675717]])

In [5]:
X_scaled[0]

array([0.87993796, 0.93574309, 0.92082308, 1.00759047, 0.33292613,
       0.93900401])

In [5]:
# Normaliza a imagem de teste
# X_scaled = X_scaled / 65535.0 # 16 bits
# ATENÇÃO: para a imagem landsat-5, não precisa normalizar
#X_scaled = X_scaled / 255.0 # 8 bits

# reshape
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
print(X_scaled.shape)

(180000, 1, 6)


In [6]:
# Define the parameters of the model
model5 = keras.models.load_model('modelos/my_model_allb_5.h5')
model10 = keras.models.load_model('modelos/my_model_allb_10.h5')
model20 = keras.models.load_model('modelos/my_model_allb_20.h5')
model25 = keras.models.load_model('modelos/my_model_allb_25.h5')
model100 = keras.models.load_model('modelos/my_model_allb_100.h5')


2023-04-25 02:57:24.994774: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-25 02:57:24.994957: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 02:57:24.995000: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-04-25 02:57:24.995041: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-04-25 02:57:24.995081: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [7]:
predict5 = model5.predict(X_scaled)
predict10 = model10.predict(X_scaled)
predict20 = model20.predict(X_scaled)
predict25 = model25.predict(X_scaled)
predict100 = model100.predict(X_scaled)

In [8]:
visualiza5 = predict5[:,1]
visualiza10 = predict10[:,1]
visualiza20 = predict20[:,1]
visualiza25 = predict25[:,1]
visualiza100 = predict100[:,1]

In [9]:
visualiza25

array([7.0951559e-02, 4.5889628e-06, 1.7143287e-04, ..., 2.7436770e-25,
       1.2831620e-25, 2.7858312e-25], dtype=float32)

In [21]:
valor_corte = 0.30
arr_ep = ['5', '10', '20', '25']
arr_vis = [visualiza5, visualiza10, visualiza20, visualiza25]

for i in range(4):
    resultPredict = (arr_vis[i]>valor_corte).astype(int)
    arrayOfnumeros = np.array(resultPredict)
    # (360, 500) - lagoa v1
    # (200, 150) - lagoa v2
    # (300, 650) - barra
    arrayOfnumeros = np.reshape(arrayOfnumeros, (360, 500))
    data = im.fromarray((arrayOfnumeros*255).astype(np.uint8))
    data.save(f'images/output/rn_epochs/{arr_ep[i]}_epochs/allb_{str(valor_corte).replace(".", "_")}.png')

In [11]:
valor_cortes = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99, 0.999]
# valor_cortes = [0.99, 0.995, 0.999, 0.9995, 0.9999]
total_cortes = 12
visualiza = [
    ["5", visualiza5],
    ["10", visualiza10],
    ["20", visualiza20],
    ["25", visualiza25],
    ["100", visualiza100]
]
for v in visualiza:
    for j in range(total_cortes):
        resultPredict = (v[1]>valor_cortes[j]).astype(int)
        arrayOfnumeros = np.array(resultPredict)
        # (360, 500) - lagoa v1
        # (200, 150) - lagoa v2
        # (300, 650) - barra
        arrayOfnumeros = np.reshape(arrayOfnumeros, (360, 500))
        data = im.fromarray((arrayOfnumeros*255).astype(np.uint8))
        #data.save(f'images/output/rn_epochs/land_5/{v[0]}_epochs/b_all_{str(valor_cortes[j]).replace(".", "_")}_t2.png')
        data.save(f'images/output/rn_epochs/{v[0]}_epochs/b_all_{str(valor_cortes[j]).replace(".", "_")}_sn.png')